This notebook provide a technical background of the multi-family apartment building model.

# Apartment Model

The thermal model is a based on a four-zone ETP model, where the following zones variables are defined:

Table 1: Zone variables

| Zone Name            | Temperature | Mass   | Heat   |
| :------------------- | :---------: | :----: | :----: |
| Occupied apartment   | $T_A$       | $C_A$  | $Q_A$  |
| Unoccupied apartment | $T_U$       | $C_U$  | $Q_U$  |
| Building core        | $T_C$       | $C_C$  | $Q_C$  |
| Building mass        | $T_M$       | $C_M$  |   -    |
| Outdoor air          | $T_O$       |   -    |   -    |

The model assumes that every zone is connected to every other zone with the following zone-zone conductance parameters defined

Table 2: Zone-zone conductance

| Conductance | Connected zones                       |
| ----------- | ------------------------------------- |
| $U_{OA}$    | Occupied apartment & outdoor air      |
| $U_{OU}$    | Unoccupied apartment & outdoor air    |
| $U_{OC}$    | Building core & outdoor air           |
| $U_{OM}$    | Building mass & outdoor air           |
| $U_{AU}$    | Occupied & unoccupied apartments      |
| $U_{AC}$    | Occupied apartments & building core   |
| $U_{AM}$    | Occupied apartments & building mass   |
| $U_{UC}$    | Uncccupied apartments & building core |
| $U_{UM}$    | Unoccupied apartments & building mass |
| $U_{CM}$    | Building core & building mass         |

The thermal dynamics are defined by Equation (1)

$$
    \begin{bmatrix} 
        \dot T_A 
    \\ 
        \dot T_U 
    \\
        \dot T_C
    \\
        \dot T_M
    \end{bmatrix}
    =
    \begin{bmatrix}
        -\frac{U_{OA}+U_{AU}+U_{AC}+U_{AM}}{C_A} & \frac{U_{AU}}{C_A} & \frac{U_{AC}}{C_A} & \frac{U_{AM}}{C_A}
    \\
        \frac{U_{AU}}{C_U} & -\frac{U_{OA}+U_{AU}+U_{UC}+U_{UM}}{C_U} & \frac{U_{UC}}{C_U} & \frac{U_{UM}}{C_U}
    \\
        \frac{U_{AC}}{C_C} & \frac{U_{UC}}{C_C} & -\frac{U_{OC}+U_{AC}+U_{UC}+U_{CM}}{C_C} & \frac{U_{CM}}{C_C}
    \\
        \frac{U_{AM}}{C_M} & \frac{U_{UM}}{C_M} & \frac{U_{CM}}{C_M} & -\frac{U_{OM}+U_{AM}+U_{UM}+U_{CM}}{C_M}
    \end{bmatrix}
    \begin{bmatrix}
        T_A
    \\
        T_U
    \\
        T_C
    \\
        T_M
    \end{bmatrix}
    +
    \begin{bmatrix}
        \frac{1}{C_A} & 0 & 0 & \frac{U_{OA}}{C_A}
    \\
        0 & \frac{1}{C_U} & 0 & \frac{U_{OU}}{C_U}
    \\
        0 & 0 & \frac{1}{C_C} & \frac{U_{OC}}{C_C}
    \\
        0 & 0 & 0 & \frac{U_{OM}}{C_M}
    \end{bmatrix}
    \begin{bmatrix}
        Q_A
    \\
        Q_U
    \\
        Q_C
    \\
        T_O
    \end{bmatrix}
    \qquad (1)
$$
where the heat gains are
- $Q_A = Q_{AH} + Q_{AS} + Q_{AV} + Q_{AE}$,
- $Q_U = Q_{UH} + Q_{US}$
- $Q_C = Q_{CH} + Q_{CS} + Q_{CV}$

The heat gains for each zones are defined as follows

Table 3: Occupied apartment heat gains

| Heat source      | Parameter |
| ---------------- | --------- |
| HVAC             | $Q_{AH}$  |
| Solar gain       | $Q_{AS}$  |
| Ventilation gain | $Q_{AV}$  |
| End-use gains    | $Q_{AE}$  |

Table 4: Unoccupied apartment heat gains 

| Heat source      | Parameter |
| ---------------- | --------- |
| HVAC             | $Q_{UH}$  |
| Solar gain       | $Q_{US}$  |

Table 5: Building core heat gains

| Heat source      | Parameter |
| ---------------- | --------- |
| HVAC             | $Q_{CH}$  |
| Solar gain       | $Q_{CS}$  |
| Ventilation gain | $Q_{CV}$  |


The following end-uses contribute to heat gain in building zones and are driven by schedules

Table 6: Zone end-use schedule source

| End-use     | $Q_{AE}$      | $Q_{CE}$     |
| ----------- | ------------- | ------------ |
| Lights      | `lighting`    | `hallway`    |
| Plugs       | `plugs`       |      -       |
| Cooking     | `cooking`     |      -       |
| Dishwashing | `dishwashing` |      -       |
| Washing     | `washing`     |      -       |
| Drying      | `drying`      |      -       |
| Hotwater    | `hotwater`    |      -       |


Equation (1) is represented canonically as

$$
    \boxed { \dot {\textbf T} = \textbf A \textbf T + \textbf B_1 \textbf q + \textbf B_2 \textbf u } \qquad (2)
$$

where $\textbf q$ are the ambient heat gains/losses, and $\textbf u$ is the controlled heat gains/losses, with

$$
    \textbf B_1 = \begin{bmatrix}
        {U_{OA}} \over {C_A} & 1 \over {C_A} & 1 \over {C_A} & 1 \over {C_A} & 0 & 0 & 0 & 0 
    \\
        {U_{OU}} \over {C_U} & 0 & 0 & 0 & 1 \over {C_U} & 0 & 0 & 0 
    \\
        {U_{OC}} \over {C_C} & 0 & 0 & 0 & 0 & 1 \over {C_C} & 1 \over {C_C} & 1 \over {C_C}
    \\
        {U_{OM}} \over {C_M} & 0 & 0 & 0 & 0 & 0 & 0 & 0 
    \end{bmatrix}
    \qquad \textrm{and} \qquad
    \textbf B_2 = \begin{bmatrix}
        1 \over {C_A} & 0 & 0
    \\
        0 & 1 \over {C_U} & 0 
    \\
        0 & 0 & 1 \over {C_C}
    \\
        0 & 0 & 0
    \end{bmatrix}
$$

and

$$
    \textbf q = \begin{bmatrix} T_O \\ Q_{AS} \\ Q_{AV} \\ Q_{AE} \\ Q_{US} \\ Q_{CS} \\ Q_{CV} \\ Q_{CE} \end{bmatrix}
    \qquad \textrm{and} \qquad
    \textbf u = \begin{bmatrix} Q_{AH} \\ Q_{UH} \\ Q_{CH} \end{bmatrix}.
$$

## HVAC Controls

The following thermostat setpoints are use to control HVAC

| Zone       | $T_{AH}$ | $T_{AC}$ |
| ---------- | -------- | -------- |
| Occupied   | $T_{AH}$ | $T_{AC}$ |
| Unccupied  | $T_{UH}$ | $T_{UC}$ |
| Core       | $T_{CH}$ | $T_{CC}$ |

# Heating/Cooling Solution Method

The model assumes that the setpoints are maintained in the controlled zone, and computes the heating/cooling load required by setting Equation (2) equal to zero.  However, if the heating/cooling load exceeds the capacity of the central plant or the HVAC units, then the power will be saturated, and the Equation (2) will provide the temperature changes, if any, for each zone.

## Step 1 - Determine the system mode

Solve Equation (2) for $\textbf{T}$ given $\dot {\textbf T}$ and $\textbf{u} = 0$. When $\textbf{A}$ is full rank, the solution is

$$
    \textbf{T}^o = - \textbf{A}^{-1} \textbf{B}_1 \textbf{q}
$$

## Step 2 - Compute the equilibrium load

Solve Equation (2) for the augmented $\textbf{u}$ at the temperature setpoint given $\dot {\textbf T}=0$, i.e.,

$$
    \textbf{u} = -\textbf{B_2}^+ ( \textbf{A} \textbf T^* + \textbf{B_1} \textbf{q} )
$$

where the pseudo-inverse $\textbf{B_2}^+ = (\textbf{B_2}^\mathrm{T} \textbf{B_2})^{-1} \textbf{B_2}^\mathrm{T})$,
with the setpoint

$$
    \textbf T^* = T_{set} - m \begin{bmatrix}  0 \\ T_{unoccupied} \\ T_{core} \\ 0 \end{bmatrix}
$$

where 

- $T_{set}$ is the general setpoint temperature for the building,
- $m$ is the building operating mode, i.e., `-1` for cooling and `+1` for heating,
- $T_{unoccupied}$ is the unoccupied temperature setpoint offset, e.g., `10 degF`, and
- $T_{core}$ is the core temperature setpoint offset, e.g., `5 degF`.

## Step 3 - Constrain the heat gain/loss

The value of $\textbf u$ may exceed the limits of the HVAC system, in which can they must be constrained such that element-wise

$$
    \textbf u^* = \textbf u \qquad \mathrm{s.t} \qquad \underline {\textbf u} \le {\textbf u} \le \overline {\textbf u}
$$

where $\underline {\textbf u}$ is the maximum cooling capacity and $\overline {\textbf u}$ is the maximum heating capacity of the HVAC system.  

## Step 4 - Compute the temperature change

Given the constrained values $\textbf u^*$, we update the zone temperatures using Equation (2):

$$
    \dot {\textbf T} = \textbf{A} \textbf{T} + \textbf B_1 \textbf q + \textbf B_2 \textbf u^*
$$

where

$$
    \textbf T = \begin{bmatrix} T_A \\ T_U \\ T_C \\ T_M \end{bmatrix}
$$

# Electric Power Demand

The power consumption is the sum of all the end-use loads and the HVAC load given the efficiency of the HVAC systems:

$$
    P = Q_{AE} + Q_{CE} + { 1\over \eta_M } ||u||_1^*
$$

where $\eta_M$ is the efficiency of the HVAC system in the mode $M$.

# Thermal Parameters

The following parameters are used to compute the thermal parameters for a building.

Table 3: Building design parameters

| Parameter                  | Description
| -------------------------- | -------------------------------------------
| $N$                        | Number of floors (integer)
| $M$                        | Number of units per floor (integer)
| $K_X$                      | Exterior core (no=0, yes=1)
| $K_D$                      | Core loading (0=one side, 1=both sides)
| $F$                        | Floor thickness (ft)
| $X$                        | Unit width (ft)
| $Y$                        | Unit depth (ft)
| $Z$                        | Unit height (ft)
| $W$                        | Interior core width (ft)
| $A_W$                      | Unit window area (sf)
| $A_D$                      | Unit door area (sf)
| $C_{int}$                  | Unit interior mass (Btu/degF)
| $S_{floor}$                | Floor specific mass (Btu/degF.sf)

Table 4: Building thermal properties

| Parameter    | Description
| ------------ | ---------------------
| $R_{ext}$    | Exterior wall R-value
| $R_{int}$    | Interior wall R-value
| $R_{window}$ | Window R-value
| $R_{door}$   | Unit door R-value
| $R_{mass}$   | Mass R-value (i.e., floor, ceiling, furnishings)

Table 5: Derived building properties

| Parameter | Calculation &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        | Description
| --------- | ------------------- | -------------------------------------------
| $A$       | $ X Y $             | Unit floor area (sf)
| $A_U$     | $ Y Z $             | Interior unit-unit wall area (sf)
| $A_O$     | $ X Z - A_W $       | Exterior unit wall area (sf)
| $A_C$     | $ Z X - A_D $       | Interior unit-core wall area (sf)
| $V$       | $ X Y Z $           | Unit volumne (cf)

To compute the thermal parameters we identify the expore areas for each zone. The occupancy factor $\beta$ is the ratio of occupied units to the total number of units.

Table 6: Conductance calculations

| Conductance | Parameter calculation &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
| ----------- | ---------------------------------------
| $U_{OA}$    | $ N M \beta (K_X+1) \left( { {A_O} \over {R_{ext}} } + { {A_W} \over {R_{window}} } \right) $
| $U_{OU}$    | $ N M (1-\beta) (K_X+1) \left( { {A_O} \over {R_{ext}} } + { {A_W} \over {R_{window}}} \right) $
| $U_{OC}$    | $ N (1-K_X) [ 2 W Z + (K_D+1) M X ] \left( { {A_O} \over {R_{ext}} } + { {A_W} \over {R_{window}} } \right) $
| $U_{OM}$    | $ N {{F (MX+2(K_D+1)Y+2W)} \over {R_{ext}}} $
| $U_{AU}$    | $ N M \beta (1-\beta) {A_U} \over {R_{int}} $
| $U_{AC}$    | $ N M \beta \left( { {A_W} \over {(1-K_X) R_{int} + K_X R_{ext}}} + { {A_D} \over {R_{door}} } \right) $
| $U_{AM}$    | $ N M \beta {A \over {R_{mass}}} $
| $U_{UC}$    | $ N M (1-\beta) \left( { {A_W} \over {(1-K_X) R_{int} + K_X R_{ext}}} + { {A_D} \over {R_{door}} } \right) $
| $U_{UM}$    | $ N M (1-\beta) { A \over {R_{mass}} } $
| $U_{CM}$    | $ N W M { {K_D+1} \over {R_{floor}}} $

Table 7: Capacitance calculation

| Heat capacity | Parameter calculation &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
| ------------- | -------------------------------------
| $C_A$         | $ N M \beta ( V S_{air} + C_{int} ) $
| $C_U$         | $ N M (1-\beta) ( V S_{air} + C_{int}) $
| $C_C$         | $ N M X W H C_{air} $
| $C_M$         | $ N M S_{floor} ( A + K X W )  $

# Example

The following example illustrates this method.  The method requires a linear algebra library:

In [1]:
from numpy import *

The outdoor temperature is assumed to be 20F below the indoor setpoint.

In [2]:
# set outdoor temperature (delta from T_A and T_M)
T_O = -20 # computed from weather data relative the apartment setpoint (e.g., T_A = 40 degF - T_O)

The building has the following characteristics

In [3]:
# Table 3: Building design parameters
N = 10 # 10 floors
M = 20 # 20 units per floor
K_X = 0 # corridor is not exterior
K_D = 1 # corridor is double-sided
F = 2 # floor thickness (ft)
X = 30 # unit width (ft)
Y = 40 # unit depth (ft)
Z = 7.5 # unit height (ft)
W = 4.5 # corridor width (ft)
omega = 0.5 # exterior window/wall ratio
A_D = 3*6.6 # unit door area (sf)
C_int = 2000 # occupied unit interior thermal capacitance (Btu/degF)
S_floor = 150 # density of concrete floor (Btu/cf)

In [4]:
# Table 4: Building thermal properties
R_ext = 19
R_int = 5
R_window = 2
R_door = 3
R_mass = 1
R_floor = 1

In [5]:
# Table 5: Derived building properties
A_W = omega*X*Z # unit window area (sf)""
A_F = X*Y # unit floor area (ft)
A_U = Y*Z # unit-unit wall area (ft)
A_O = X*Z - A_W # unit-outdoor wall area (ft)
A_C = X*Z - A_D # unit-core wall area (ft)
V = X*Y*Z # unit volume (cf)
S_air = 0.24 * 0.074 # specific heat x density of air (Btu/degF.cf)

In [6]:
# Building occupancy
beta = 0.9 # fraction of units that are occupied

In [7]:
# set zone heat gains
Q_AS = 500000 # solar gains to occupied apartments (computed from weather data)
Q_AV = -10000 # ventilation gains to occupied apartments (computed from weather data)
Q_AE = 10000 # electric load gains to occupied aparments (computed from equipment data)
Q_US = (1-beta)*Q_AS # solar gains to unoccupied apartments (computed from weather data)
Q_CS = 1000 # solar gains to core spaces (computed from weather data)
Q_CV = -5000 # ventilation gains to core spaces (computed from weather data)

In [8]:
# Building mode
m = -sign(T_O) # -1 for cooling, +1 for heating

The HVAC system capabilities are

In [9]:
# set heat/cool constraints
u_min = matrix([[-500000],[-200000],[-1000000],[0]])
u_max = matrix([[500000],[200000],[1500000],[0]])
# plant efficiency
e_cool = 3.0
e_heat = 4.0
# temperature setpoints
T_unoccupied = 10 # unoccupied zone temperature setpoint offset (degF)
T_core = 5 # core zone temperature setpoint offset (degF)

In [10]:
# Table 6: Conductance calculations
U_OA = N * M * beta * (K_X+1) * ( A_O/R_ext + A_W/R_window )
U_OU = U_OA * (1-beta) / beta
U_OC = N * (1-K_X) * (2*W*Z+(1-K_D)*M*X) * ( A_O/R_ext + A_W/R_window )
U_OM = N * F * (M*X+2*(K_D+1)*Y+2*W) / R_ext
U_AU = N * M * beta * (1-beta) * A_U/R_int
U_AC = N * M * beta * ( (A_W/((1-K_X)*R_int+K_X*R_ext)) + (A_D/R_door) ) 
U_AM = N * M * beta * A_F/R_mass
U_UC = N * M * (1-beta) * ( (A_W/((1-K_X)*R_int+K_X*R_ext)) + (A_D/R_door) )
U_UM = N * M * (1-beta) * A_F/R_mass
U_CM = N * M * W * (K_D+1)/R_floor
print("Table 6: Conductance calculations (kBtu/degF.h)\n\n" + 
      "U_OA U_OU U_OC U_OM U_AU U_AC U_AM U_UC U_UM U_CM\n" +
      "---- ---- ---- ---- ---- ---- ---- ---- ---- ----\n" +
      "%4.0f %4.0f %4.0f %4.0f %4.0f %4.0f %4.0f %4.0f %4.0f %4.0f" 
      % (U_OA/1000,U_OU/1000,U_OC/1000,U_OM/1000,U_AU/1000,
         U_AC/1000,U_AM/1000,U_UC/1000,U_UM/1000,U_CM/1000))

Table 6: Conductance calculations (kBtu/degF.h)

U_OA U_OU U_OC U_OM U_AU U_AC U_AM U_UC U_UM U_CM
---- ---- ---- ---- ---- ---- ---- ---- ---- ----
  11    1   42    1    1    5  216    1   24    2


In [11]:
# Table 7: Capacitance calculations
C_A = N * M * beta * V * S_air
C_U = N * M * (1-beta) * V * S_air
C_C = N * M * X * W * Z * S_air
C_M = N * M * (C_int + (1 + X * W) * F * S_floor)
print("Table 7: Capacitance calculations (kBtu/degF)\n\n"
     + "C_A  C_U  C_C  C_M\n"
     + "---- ---- ---- ----\n"
     + "%4.0f %4.0f %4.0f %4.0f"
     % (C_A/1000,C_U/1000,C_C/1000,C_M/1000))

Table 7: Capacitance calculations (kBtu/degF)

C_A  C_U  C_C  C_M
---- ---- ---- ----
  29    3    4 8560


In [12]:
# boundary condition
q = matrix([[T_O], [Q_AS+Q_AV+Q_AE], [Q_US+Q_CS], [Q_CV]])
print("Thermal conditions\n\n"
     + "T_O   Q_A  Q_U  Q_C\n"
     + "degF     kBtu/h    \n"
     + "---- ---- ---- ----")
print("%4.0f %4.0f %4.0f %4.0f"
     % (q[0],q[1]/1000,q[2]/1000,q[3]/1000))

Thermal conditions

T_O   Q_A  Q_U  Q_C
degF     kBtu/h    
---- ---- ---- ----
 -20  500   51   -5


In [13]:
from numpy.linalg import *
# compute the solution matrices
A = matrix([ 
      [-(U_OA+U_AU+U_AC+U_AM)/C_A,                   U_AU/C_A,                   U_AC/C_A,                   U_AM/C_A],
      [                  U_AU/C_U, -(U_OU+U_AU+U_UC+U_UM)/C_U,                   U_UC/C_U,                   U_UM/C_U],
      [                  U_AC/C_C,                   U_UC/C_C, -(U_OC+U_AC+U_UC+U_CM)/C_C,                   U_CM/C_C],
      [                  U_AM/C_M,                   U_UM/C_M,                   U_CM/C_M, -(U_OM+U_AM+U_UM+U_CM)/C_M]
    ])
a = 0.9 # fraction of heat gain that goes to zone instead of mass
b = 1-a # fraction of heat gain that is taken up directly by mass
B1 = matrix([ 
       [ U_OA/C_A, a/C_A,     0,     0],
       [ U_OU/C_U,     0, a/C_U,     0],
       [ U_OC/C_C,     0,     0, a/C_C],
       [ U_OM/C_M, b/C_M, b/C_M, b/C_M]
      ])
B2 = matrix([ 
       [ a/C_A,     0,     0,     0],
       [     0, a/C_U,     0,     0],
       [     0,     0, a/C_C,     0],
       [ b/C_M, b/C_M, b/C_M, 1/C_M]
      ])
ndim(B2)

2

In [14]:
T = matrix([[0],[0],[0],[0]])
Ainv = inv(A)
T_eq = -Ainv*(B1*q)
dT = A*T_eq + B1*q
T_set = m*matrix([[0],[T_unoccupied],[T_core],[0]])
print("Temperatures (degF)\n\n"
     + "Zone  dT/dt   Eq   Set \n"
     + "----- ----- ----- -----")
print("%-5.5s %5.1f %5.1f %5.1f" % ('T_A',dT[0],T_eq[0],T_set[0]))
print("%-5.5s %5.1f %5.1f %5.1f" % ('T_U',dT[1],T_eq[1],T_set[1]))
print("%-5.5s %5.1f %5.1f %5.1f" % ('T_C',dT[2],T_eq[2],T_set[2]))
print("%-5.5s %5.1f %5.1f   --" % ('T_M',dT[3],T_eq[3]))

Temperatures (degF)

Zone  dT/dt   Eq   Set 
----- ----- ----- -----
T_A    -0.0   7.9   0.0
T_U    -0.0   7.8  10.0
T_C     0.0 -15.8   5.0
T_M     0.0   7.9   --


In [15]:
# initial temperature
T = matrix([[0], [0], [0], [0]])
# solve for the required control
u_req = -inv(B2) * (A*T_set+B1*q);
# constrain the solution
u = hstack((u_max,hstack((u_req,u_min)).max(axis=1))).min(axis=1)
# compute the temperature changes
dT = A*T + B1*q + B2*u
# compute time to change 0.1 degF
dt = 3600/max(abs(dT))[0,0]
# show me
print("shortfall = %.0f kBtu/h" % (sum(u_req - u)/1000))
print("dTA = %.2f degF/h" % (dT[0,0]))
print("dTU = %.2f degF/h" % (dT[1,0]))
print("dTC = %.2f degF/h" % (dT[2,0]))
print("dTM = %.2f degF/h" % (dT[3,0]))
print("dt = %.0f sec" % (dt))

shortfall = -334 kBtu/h
dTA = -1.29 degF/h
dTU = 62.89 degF/h
dTC = 67.32 degF/h
dTM = 0.02 degF/h
dt = 53 sec


In [16]:
# power requirements
if m < 0: eta = e_heat 
else: eta = e_cool
P = m*sum(u)/eta
print("Mode Power\n"
      + "---- -----\n"
      + "%4.0f %5.0f"
      % (m,P/1000))

Mode Power
---- -----
   1   371
